# OpenUVF Crack Detector v1








Code adapted from object_detection_tutorial.ipynb by The TensorFlow Authors.


Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

## Imports

In [1]:
#Import Essential Packages
import numpy as np
import os
import six.moves.urllib as urllib
import sys
#import tarfile
import tensorflow as tf
#import zipfile
import time
import logging
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

#Import Object Detection Utils
from core import object_detection as object_detection
from core.object_detection.utils import ops as utils_ops
from core.object_detection.utils import label_map_util
from core.object_detection.utils import visualization_utils as vis_util
from core.utils.detection_utils import image_pipeline
from core.utils.detection_utils import assemble_module
from core.utils.detection_utils import initialize_statistics_dict
from core.utils.detection_utils import log_statistics
from core.utils.detection_utils import build_visualization

#Ensure up to date version
if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


C:\Users\RSDG\Documents\OpenUVF\core\object_detection\utils\visualization_utils.py:27: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


## Directories

Specify here the input and output directories.

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.

Inputs:
  1. path_to_frozen_graph (str): Path to the frozen detection graph.pb file representing the model you wish you to use.
  2. path_to_labels (str): Path to the label map defined for your model.
  3. images_dir (str): Path to the directory containing the images you wish to evaluate. Default value should be 'inputs/detection'
  4. output_dir (str): Path to the directory where you wish the outputs to be saved. Default value should be 'outputs/detection'

In [2]:
path_to_frozen_graph = 'core/models/C1C1(FRCNN_RN101_COCO)_v2/frozen_inference_graph.pb'
path_to_labels = os.path.join('core/labels', '1_Class_label_map.pbtxt')
images_dir = 'inputs/detection'
output_dir = 'outputs/detection'

## Function Setup



##### Pipeline Settings:
The pipeline defines how the images are processed. The pipeline is broken down into groups and batches. Groups are images loaded into memory for processing, and is accordingly RAM limited. Batches are images fed to the detector and is primarily VRAM limited (if using a gpu).
 1. pipeline_type (str): defines the mode used for processing images. 
      - Options:
          1. modules-cells: performs cell level detection 
          2. modules
          3. cells

In [3]:
# Environment Setup
%matplotlib inline 

# Function Settings
display = True
shard_images = True
shard_images_count = 72

#Pipeline Settings
pipeline_type = 'modules-cells'
max_group_size = 72
max_batch_size = 72


#Optional Settings


## Input Management

In [4]:
images_list = image_pipeline(pipeline_type, images_dir, shard_images_count)

[72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 3, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 

## Model preparation 

## Load the frozen Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(path_to_frozen_graph, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
def run_inference(np_images, graph, tensor_dict, image_tensor):
      
    #Defining Storage Array
    outputs = []

    #Tracking image count
    nim = 1

    #Iterate through directory
    #for image_name in os.listdir(images_path):
    for image_np in np_images:

        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)                         

        # Run inference
        output = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image_np, 0)})

        # all outputs are float32 numpy arrays, so convert types as appropriate
        output['num_detections'] = int(output['num_detections'][0])
        output['detection_classes'] = output[
          'detection_classes'][0].astype(np.uint8)
        output['detection_boxes'] = output['detection_boxes'][0]
        output['detection_scores'] = output['detection_scores'][0]
        if 'detection_masks' in output:
            output['detection_masks'] = output['detection_masks'][0]

        #Storing output
        outputs.append(output)

        #Updating image number
        nim = nim + 1
                
    return outputs

## Crack Statistics 

This function accumulates crack statistics to allow better understanding of 

including:
    1.  plant_num_cells (int) = total cells considered over the plant
    2.  plant_num_modules (int) = total modules considered over the plant
    3.  plant_num_cracks (int) = total cracks detected over the plant
    4.  plant_num_cracks_per_cell (int) = average cracks per cell over the plant
    5.  plant_num_cracks_per_cracked_cell (int) = average cracks per cracked cell over the plant
    6.  plant_num_cracks_per_module (int) = average cracks per module over the plant
    7.  plant_num_cracks_per_cell_index (1xnCells ndarray) = total number of cracks for each cell index over the plant
    8.  module_num_cells (dict) = number of cells for each module considered
    9.  module_num_cracks (dict) = number of cracks for each module considered
    10. module_num_cracks_per_cell (list) = number of cracks
    11. module_num_cracks_per_cell_index (list of 1xnCells ndarrays) = number of cracks for each module considered in each cell index
    12. cell_num_cracks (list) = number of cracks in the cell
    13. cell_crack_area (list of lists) = area for each crack in a cell
    
    
    
    
    2. total number of cracked cells 9
    3. average cracks per cracked cells
    4. average cracks per all cells
    5. total number of cracks per module
    6. total number of cracked cells per module 
    7. total of cracks per cell index (to identify if there is a localization trend)
    
    returns statistics dictionary
    
    assumes outputs and image_list correspond 1 to 1


In [ ]:
def output_statistics(statistics):
    print('Plant:')
    print('  # Modules               = ' + str(statistics['plant_num_modules']))
    print('  # Cells                 = ' + str(statistics['plant_num_cells']))
    print('  # Cracks                = ' + str(statistics['plant_num_cracks']))
    print('  # Cracked Cells         = ' + str(statistics['plant_num_cracked_cells']))
    print('  # Cracks/Cell           = ' + str(statistics['plant_num_cracks_per_cell']))
    print('  # Cracks/Cracked-Cell   = ' + str(statistics['plant_num_cracks_per_cracked_cell']))
    print('  # Cracks/Module         = ' + str(statistics['plant_num_cracks_per_module']))
    
    
    
        
        
        
        
        

## Outputting Images

In [ ]:
def output_images(np_images, image_list):

    for i in range(len(image_list)):

        #Pull image and name
        image_np = np_images[i]        
        image_name = image_list[i]
        
        print(type(image_np))
        #Save Image to output directory
        image_out = Image.fromarray(image_np)
        image_out.save(os.path.join(output_dir, image_name))
        

## Main

In [ ]:
# Defining variables
statistics = initialize_statistics_dict()

# Optional inputs
probability_thresh = .5


#Execution Tracking
execution_times = dict(full=0, batch=[])
full_start_time = time.time()

# Get handles to input and output tensors
graph = detection_graph
with graph.as_default():
    with tf.Session() as sess:
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
            'num_detections', 'detection_boxes', 'detection_scores',
            'detection_classes', 'detection_masks'
        ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                    tensor_name)
        if 'detection_masks' in tensor_dict:
            # The following processing is only for single image
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
            detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
            
            # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
            real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
            detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
            detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
            detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                detection_masks, detection_boxes, image.shape[0], image.shape[1])
            detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)
            
            # Follow the convention by adding back the batch dimension
            tensor_dict['detection_masks'] = tf.expand_dims(
                detection_masks_reframed, 0)
            
        #Define Image Tensor
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')   
        
        #TEMPORARY - DEFINE IMAGE SIZE - FUTURE USE SIZE FROM PIPELINE
        image_size = (150, 150)
        
        #Iterate through images shards
        batch = 1
        for image_list in images_list:
            
            #Initialize time tracking
            batch_start_time = time.time()
            
            #User feedback
            print('Batch ' + str(batch) + ' Processing...')

            #Load the images
            np_images = []
            for image_name in image_list:

                #Load the image and convert to 
                image_path = os.path.join(images_dir, image_name)
                image = Image.open(image_path).resize(image_size)
                image_np = load_image_into_numpy_array(image)

                #Append it to array
                np_images.append(image_np)           

            
            #Run Detection for Image Set        
            start_time = time.time()
            outputs = run_inference(np_images, detection_graph, tensor_dict, image_tensor) 
            
            #Log Statistics
            statistics = log_statistics(statistics, probability_thresh, outputs, image_list, np_images)
            
            #Assemble Module
            np_images, image_list, outputs = assemble_module(np_images, image_list, outputs)
            
            #Generate Visualizations on images
            np_images_labeled = build_visualization(np_images, outputs, category_index, probability_thresh)
            
            #Saving the images
            output_images(np_images_labeled, image_list)
            
            #Feedback
            print('Batch ' + str(batch) + ' Processed successfully!')
            
            #Count which batch
            batch+=1
            
            #Log Processing time
            execution_times['batch'].append(time.time() - batch_start_time)
            

#Log total process time
execution_times['full'] = time.time() - full_start_time
            
#Outputs
output_statistics(statistics)
print('Total Execution Time =         ' + str(execution_times['full']) + ' s')
print('Mean Batch Execution Time =    ' + str(sum(execution_times['batch'])/float(len(execution_times['batch']))) + ' s')



Batch 1 Processing...
<class 'numpy.ndarray'>
Batch 1 Processed successfully!
Batch 2 Processing...
<class 'numpy.ndarray'>
Batch 2 Processed successfully!
Batch 3 Processing...
<class 'numpy.ndarray'>
Batch 3 Processed successfully!
Batch 4 Processing...
<class 'numpy.ndarray'>
Batch 4 Processed successfully!
Batch 5 Processing...
<class 'numpy.ndarray'>
Batch 5 Processed successfully!
Batch 6 Processing...
<class 'numpy.ndarray'>
Batch 6 Processed successfully!
Batch 7 Processing...
<class 'numpy.ndarray'>
Batch 7 Processed successfully!
Batch 8 Processing...
<class 'numpy.ndarray'>
Batch 8 Processed successfully!
Batch 9 Processing...
<class 'numpy.ndarray'>
Batch 9 Processed successfully!
Batch 10 Processing...
<class 'numpy.ndarray'>
Batch 10 Processed successfully!
Batch 11 Processing...
<class 'numpy.ndarray'>
Batch 11 Processed successfully!
Batch 12 Processing...
<class 'numpy.ndarray'>
Batch 12 Processed successfully!
Batch 13 Processing...
<class 'numpy.ndarray'>
Batch 13 Pr

## Outputting Images

## Crack Visualization

Optional Inputs:
  1. max_figures (int): specifies the max number of pyplot figures that can be generated
  2. images_sampled (int, int array, or str): specifies which images are displayed. Options:
     1. 'random' - displays random images until max_figures is reached
     2. 'first' - displays first set of images until max_figures is reached
      

In [ ]:
#Visualization function
def visualize_images(images_list, outputs, max_figures=100, images_sampled='random', figure_size=(5,5)):

    
    #Defining images to display
    image_ct = len(output)
    

    #Visualize and save images
    i = 0;
    for image_name in images_list:

        #Pull Image and VisBox Data
        output = outputs[i]
        image_path = os.path.join(images_dir, image_name)
        image = Image.open(image_path)
        image_np = load_image_into_numpy_array(image)

        #Apply Visualization Boxes with labels
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output['detection_boxes'],
            output['detection_classes'],
            output['detection_scores'],
            category_index,
            instance_masks=output.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=2)

        #Display images in figure
        if i < 100 and display_output_sample:
            plt.figure(figsize=figure_size)
            plt.imshow(image_np)

        #Save Image to output directory
        image_out = Image.fromarray(image_np)    
        image_out.save(os.path.join(output_dir, image_name))

        #Increment i
        i = i + 1